In [1]:
ours2orig = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 254, 41, 42, 43, 44, 45,
46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 85,
86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103,
104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 136, 138, 132,
133, 134, 135, 220, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 185, 186,
187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201,
202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,
217, 218, 219, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232,
233, 137, 234, 235, 236, 237, 238, 239, 240, 241, 250, 251, 255, 256, 257,
258, 259, 260, 261, 262, 263, 242, 243, 244, 245, 0, 1, 2, 3, 4, 5, 6, 7, 8,
9, 10, 11, 83, 84, 131, 139, 140, 141, 181, 182, 183, 184, 246, 247, 248,
249, 252, 253]

def vec2mat(v):
    a,b = np.triu_indices(264,1)
    m = np.zeros((264,264))
    m[a,b] = v
    return m+m.T

def remap(fc, roimap=ours2orig):
    fc = fc[roimap,:]
    fc = fc[:,roimap]
    return fc

print('Complete')

Complete


In [2]:
import pickle
import numpy as np

base = '/home/anton/Documents/Tulane/Research/ImageNomer/data/PNC2/'
demo = pickle.load(open(f'{base}/demographics.pkl', 'rb'))
scans = []
a,b = np.triu_indices(264,1)
demos = [[], [], [], []]
types = ['continuous', 'categorical', 'categorical', 'categorical']

for sub in demo['age_at_cnb']:
    try:
        rac = demo['Race'][sub]
        sex = demo['Sex'][sub]
        age = demo['age_at_cnb'][sub]
    except:
        continue
    if rac not in ['AA', 'EA']:
        continue
    rac = rac == 'AA'
    sex = sex == 'M'
    for i,task in enumerate(['rest', 'nback', 'emoid']):
        try:
            s = np.load(f'{base}/conn/{sub}_task-{task}_fc.npy')
            scans.append(remap(vec2mat(s))[a,b])
            demos[0].append(age)
            demos[1].append(sex)
            demos[2].append(rac)
            demos[3].append(i)
        except:
            pass

scans = np.stack(scans)
print(scans.shape)

(3849, 34716)


In [136]:
import sys

p = '/home/anton/Documents/Tulane/Research/demo-vae/pip/src'

if p not in sys.path:
    sys.path.append(p)
    print(sys.path)

from demovae import DemoVAE

train = np.arange(200,400)

demos = [np.array(x) for x in demos]
demos_train = [x[train] for x in demos]

vae = DemoVAE(pperiod=100, use_cuda=True, nepochs=3000, latent_dim=60, weight_decay=0, lr=1e-4, loss_rec_mult=10, loss_decor_mult=10)
vae.fit(scans[train], demos_train, types)

Fitting auxilliary guidance model for demographic 0 continuous... done
Fitting auxilliary guidance model for demographic 1 categorical... done
Fitting auxilliary guidance model for demographic 2 categorical... done
Fitting auxilliary guidance model for demographic 3 categorical... done
Converting input to pytorch
Converting demographics to pytorch
Beginning VAE training
Epoch 0 ReconLoss 0.5932 CovarianceLoss 25.6168 MeanLoss 0.0867 DecorLoss 10.4852 GuidanceTargets [22.325063705444336, 1, 0, 1, 0, 0, 0, 1] GuidanceLosses ['3.6409', '0.6564', '0.6564', '0.2629', '0.2629', '2.029', '2.029', '2.029'] 
Epoch 100 ReconLoss 0.2245 CovarianceLoss 26.2042 MeanLoss 0.2092 DecorLoss 0.2749 GuidanceTargets [16.572282791137695, 1, 0, 1, 0, 0, 0, 1] GuidanceLosses ['2.9051', '1.2599', '1.2599', '3.5447', '3.5447', '0.0332', '0.0332', '0.0332'] 
Epoch 200 ReconLoss 0.1899 CovarianceLoss 25.561 MeanLoss 0.1664 DecorLoss 0.1295 GuidanceTargets [12.968782424926758, 1, 0, 1, 0, 0, 1, 0] GuidanceLosses 

DemoVAE(LR_C=100, alpha=100, bsize=1000, latent_dim=60, loss_C_mult=1,
        loss_decor_mult=10, loss_mu_mult=1, loss_pred_mult=0.001,
        loss_rec_mult=10, lr=0.0001, nepochs=3000, pperiod=100, use_cuda=True,
        weight_decay=0)

In [38]:
vae.save('/home/anton/Documents/Tulane/Research/Work/DemoVAE/models/60latpnc_reduced.torch')

print('Done')

Done


In [144]:
# Generate some synthetic data

import random

gen_demos = [[],[],[],[]]

for i in range(3000):
    age = random.randint(8,22)
    sex = random.randint(0,1)
    race = random.randint(0,1)
    task = random.randint(0,2)
    gen_demos[0].append(age)
    gen_demos[1].append(sex)
    gen_demos[2].append(race)
    gen_demos[3].append(task)

gen_demos = [np.array(x) for x in gen_demos]

xx = vae.transform(3000, gen_demos, types)
print(xx.shape)

(3000, 34716)


In [145]:
from sklearn.linear_model import Ridge

def rmse(a, b, mean=np.mean):
    return mean((a-b)**2)**0.5

test = np.arange(400,3000)

reg = Ridge(alpha=1000)
reg.fit(xx, gen_demos[0])
yhat = reg.predict(scans[test])
print(rmse(yhat, demos[0][test]))

reg = Ridge(alpha=10)
reg.fit(scans[train], demos[0][train])
yhat = reg.predict(scans[test])
print(rmse(yhat, demos[0][test]))

2.8088036578099946
2.7608210319975384


In [148]:
from sklearn.linear_model import LogisticRegression

def acc(a, b):
    return np.mean(a == b)

d = 1

reg = LogisticRegression(C=100)
reg.fit(xx, gen_demos[d])
yhat = reg.predict(scans[test])
print(acc(yhat, demos[d][test]))

reg = LogisticRegression(C=100)
reg.fit(scans[train], demos[d][train])
yhat = reg.predict(scans[test])
print(acc(yhat, demos[d][test]))

0.6161538461538462
0.6196153846153846
